<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
import folium
import pandas as pd
import branca
import geopandas as gpd
prov_geo = '../data/spain-provinces.geojson'
prov_data = '../data/series_provincias_andalucia.csv'
print(folium.__version__)
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup
import shapely.geometry

0.10.1+8.g4affacb


In [9]:
# filter to andalucia

df_geo = gpd.read_file(prov_geo)
df_geo_andalucia = df_geo[df_geo.cod_ccaa == '01']
#tesst
sevilla = shapely.geometry.Point(-5.994072,37.392529, )
df_geo_andalucia[df_geo_andalucia.contains(sevilla)]

,cod_prov,name,cod_ccaa,cartodb_id,created_at,updated_at,geometry
42,41,Sevilla,01,41,2014-09-30T00:00:00,2014-12-25T01:56:10,"MULTIPOLYGON (((-5.72508 38.19578, -5.68861 38..."


In [10]:
df = pd.read_csv(prov_data)
df.Provincia = df.Provincia.str.strip()

df_final = df[df.Fecha == df.Fecha.max()]

df_final = df_final[df_final.Provincia!='Andalucía']


dfg = df_geo_andalucia.merge(df_final, left_on='name', right_on='Provincia').drop(columns=['Provincia','Fecha'])

In [11]:
lat_range = (36.5,38.5)
lng_range = (-7.57,-1.54)
from statistics import mean

col = 'hospital'
max_val = dfg[col].max()

colormap = branca.colormap.LinearColormap(vmin=0, 
                                        vmax=max_val, 
                                        colors=['white','green','darkgreen'],
                                       caption="Ingresos acumulados")

m = folium.Map([mean(lat_range),mean(lng_range)], zoom_start=7, zoom_control=False, min_zoom=7, max_zoom=7,
               height='75%',
               width='80%',
               min_lat=lat_range[0], max_lat=lat_range[1], 
               min_lon=lng_range[0], max_lon=lng_range[1],)#, tiles='Stamen Toner')


tooltip = folium.GeoJsonTooltip(fields=['name',col], aliases=['Provincia','Ingresos acumulados'])

choropleth = folium.Choropleth(geo_data=dfg, data = dfg, columns=['name','hospital'], 
                  fill_color='BuGn',
                 key_on='feature.properties.name', 
                  highlight=True,
                  legend_name='Casos',#tooltip=tooltip,
                 )



# colormap.add_to(m)

tooltip.add_to(choropleth.geojson)

choropleth.add_to(m)
m.save('ingresos.html')
m


In [7]:
lat_range = (36.5,38.5)
lng_range = (-7.57,-1.54)
from statistics import mean

col = 'hospital'
max_val = dfg[col].max()

colormap = branca.colormap.LinearColormap(vmin=0, 
                                        vmax=max_val, 
                                        colors=['white','green','darkgreen'],
                                       caption="Ingresos acumulados")

m = folium.Map([mean(lat_range),mean(lng_range)], zoom_start=7, zoom_control=False, min_zoom=7, max_zoom=7,
               height='75%',
               width='80%',
               min_lat=lat_range[0], max_lat=lat_range[1], 
               min_lon=lng_range[0], max_lon=lng_range[1],)#, tiles='Stamen Toner')


tooltip = folium.GeoJsonTooltip(fields=['name',col], aliases=['Provincia','Ingresos acumulados'])

choropleth = folium.Choropleth(geo_data=dfg, data = dfg, columns=['name','hospital'], 
                  fill_color='BuGn',
                 key_on='feature.properties.name', 
                  highlight=True,
                  legend_name='Casos',#tooltip=tooltip,
                 )



# colormap.add_to(m)

tooltip.add_to(choropleth.geojson)

choropleth.add_to(m)
m.save('ingresos.html')
m


In [53]:
m = folium.Map(location=[35.3, -97.6], zoom_start=4)

popup=GeoJsonPopup(fields=['name','change'], aliases=['State',"% Change"], 
                   localize=True, 
                   labels=True, 
                   style="background-color: yellow;")

tooltip=GeoJsonTooltip(
    fields=["name", "medianincome", "change"],
    aliases=["State:", "2015 Median Income(USD):", "Median % Change:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)


g = folium.GeoJson(
    statesmerge,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["change"])
        if x["properties"]["change"] is not None
        else "transparent",
        "color": "black",
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
    popup=popup
).add_to(m)

colormap.add_to(m)
m

NameError: name 'statesmerge' is not defined

In [54]:
# Your code here
import folium
import branca
import geopandas
from folium.plugins import Search

print(folium.__version__)

states = geopandas.read_file(
    'https://rawcdn.githack.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json',
    driver='GeoJSON'
)

min, max = states['density'].quantile([0.05,0.95]).apply(lambda x: round(x, 2))

colormap = branca.colormap.LinearColormap(
    colors=['#f2f0f7','#cbc9e2','#9e9ac8','#756bb1','#54278f'],
    index=states['density'].quantile([0.2,0.4,0.6,0.8]),
    vmin=min,
    vmax=max
)

colormap.caption="Population Density in the United States"

m = folium.Map(location=[38,-97], zoom_start=4)


style_function = lambda x: {
    'fillColor': colormap(x['properties']['density']),
    'color': 'black',
    'weight':2,
    'fillOpacity':0.5
}


stategeo = folium.GeoJson(
    states,
    name='US States',
    style_function=style_function,
    popup=folium.GeoJsonPopup(
        fields=['name', 'density'],
        aliases=['State', 'Density'], 
        localize=True
    )
).add_to(m)


statesearch = Search(
    layer=stategeo,
    geom_type='Polygon',
    placeholder='Search for a US State',
    collapsed=False,
    search_label='name'
).add_to(m)

folium.LayerControl().add_to(m)
colormap.add_to(m)

m

0.10.1+8.g4affacb


In [ ]:
folium.features.Geo

In [ ]:
folium.plugins.